# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import * 

Here we use LiH as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [2]:
lih = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(lih)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-0.003240390291191289 [X0 Z1 Z2 Y3 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.06558452315458405 [Z6 Z8] +
0.06980180803300676 [Z6 Z9] +
0.06980180803300676 [Z7 Z8] +
0.06558452315458405 [Z7 Z9]

Fragments 2:
-0.0009084689845792826 [Y0 Z1 Z2 Z3 Z4 Z5 Y6 Y7 Z8 Z9 Z10 Y11] +
0.09902225720340917 [Z1 Z8] +
0.09654955103502388 [Z1 Z9] +
0.0683292426650509 [Z2 Z8] +
0.07512476934747551 [Z2 Z9] +
0.07512476934747551 [Z3 Z8] +
0.0683292426650509 [Z3 Z9] +
0.06009702977667614 [Z4 Z8] +
0.07068762352443517 [Z4 Z9] +
0.07068762352443517 [Z5 Z8] +
0.06009702977667614 [Z5 Z9] +
-0.21675429323818368 [Z8] +
0.07823637778985222 [Z8 Z9] +
0.06428519917726604 [Z8 Z10] +
-0.2167542932381837 [Z9] +
0.0681181856094691 [Z9 Z10]

Number of fragments: 154


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of LiH by partitioning its terms into mutually commuting fragments. 

In [3]:
comm_groups = get_commuting_group(lih)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 36
The first commuting group
-3.9344419567579023 [] +
0.007923321157849883 [X0 Y1 Y2 X3] +
-0.000155078360149252 [X0 Z1 Z2 X3 Y4 Y5] +
-0.002077886902662031 [X0 Z1 Z2 X3 Y6 Y7] +
-0.002077886902662031 [X0 Z1 Z2 X3 Y8 Y9] +
-0.0021028659073762924 [X0 Z1 Z2 X3 Y10 Y11] +
0.007923321157849883 [Y0 X1 X2 Y3] +
-0.000155078360149252 [Y0 Z1 Z2 Y3 X4 X5] +
-0.002077886902662031 [Y0 Z1 Z2 Y3 X6 X7] +
-0.002077886902662031 [Y0 Z1 Z2 Y3 X8 X9] +
-0.0021028659073762924 [Y0 Z1 Z2 Y3 X10 X11] +
0.11709373561826429 [Z0 Z3] +
-0.000155078360149252 [X1 X2 X4 X5] +
-0.002077886902662031 [X1 X2 X6 X7] +
-0.002077886902662031 [X1 X2 X8 X9] +
-0.0021028659073762924 [X1 X2 X10 X11] +
-0.000155078360149252 [Y1 Y2 Y4 Y5] +
-0.002077886902662031 [Y1 Y2 Y6 Y7] +
-0.002077886902662031 [Y1 Y2 Y8 Y9] +
-0.0021028659073762924 [Y1 Y2 Y10 Y11] +
0.11709373561826429 [Z1 Z2] +
-0.010590593747759024 [X4 X5 Y6 Y7] +
-0.01059059374775902 [X4 X5 Y8 Y9] +
-0.006733030780879654 [X4 X5 

To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [4]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999999+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [5]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-3.9344419567582896 [] +
0.1170937356182579 [X0] +
0.00015507836014926407 [X0 X1 X2 Z3 Z5] +
0.0020778869026620876 [X0 X1 X2 Z3 Z7] +
0.0020778869026620876 [X0 X1 X2 Z3 Z9] +
0.0021028659073761198 [X0 X1 X2 Z3 Z11] +
-0.007923321157850213 [X0 Z3] +
0.1170937356182579 [X1 X2] +
-0.007923321157850213 [X1 X2 Z3] +
0.00015507836014926407 [X1 X2 Z3 X4 Z5] +
0.0020778869026620876 [X1 X2 Z3 X6 Z7] +
0.0020778869026620876 [X1 X2 Z3 X8 Z9] +
0.0021028659073761198 [X1 X2 Z3 X10 Z11] +
-0.00015507836014926407 [X1 X2 X4 Z5] +
-0.0020778869026620876 [X1 X2 X6 Z7] +
-0.0020778869026620876 [X1 X2 X8 Z9] +
-0.0021028659073761198 [X1 X2 X10 Z11] +
0.08475100244618386 [X4] +
0.010590593747757936 [X4 Z5 Z7] +
0.010590593747757936 [X4 Z5 Z9] +
0.00673303078087909 [X4 Z5 Z11] +
-0.00015507836014926407 [Z5] +
0.010590593747757936 [Z5 X6 Z7] +
0.010590593747757936 [Z5 X8 Z9] +
0.00673303078087909 [Z5 X10 Z11] +
0.0782363777898527 [X6] +
0.004217284878422285 [X6 Z7 Z9] +
0.0038329864322027567 [X6 Z7 Z11] +
-0

In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [6]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-3.934441956758291 [] +
0.11709373561825798 [Z0] +
0.00015507836014926402 [Z0 Z1 Z2 Z3 Z5] +
0.0020778869026620854 [Z0 Z1 Z2 Z3 Z7] +
0.0020778869026620854 [Z0 Z1 Z2 Z3 Z9] +
0.002102865907376122 [Z0 Z1 Z2 Z3 Z11] +
-0.007923321157850214 [Z0 Z3] +
0.11709373561825798 [Z1 Z2] +
-0.007923321157850214 [Z1 Z2 Z3] +
0.00015507836014926402 [Z1 Z2 Z3 Z4 Z5] +
0.0020778869026620854 [Z1 Z2 Z3 Z6 Z7] +
0.0020778869026620854 [Z1 Z2 Z3 Z8 Z9] +
0.002102865907376122 [Z1 Z2 Z3 Z10 Z11] +
-0.00015507836014926402 [Z1 Z2 Z4 Z5] +
-0.0020778869026620854 [Z1 Z2 Z6 Z7] +
-0.0020778869026620854 [Z1 Z2 Z8 Z9] +
-0.002102865907376122 [Z1 Z2 Z10 Z11] +
0.08475100244618386 [Z4] +
0.010590593747757943 [Z4 Z5 Z7] +
0.010590593747757943 [Z4 Z5 Z9] +
0.006733030780879065 [Z4 Z5 Z11] +
-0.00015507836014926402 [Z5] +
0.010590593747757943 [Z5 Z6 Z7] +
0.010590593747757943 [Z5 Z8 Z9] +
0.006733030780879065 [Z5 Z10 Z11] +
0.078